In [ ]:
import json
import os

import riotwatcher
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("RIOT_API_KEY")
REGION = "EUW1"

lol_watcher = riotwatcher.LolWatcher(API_KEY)

In [ ]:
from tqdm import tqdm
# FILEPATH: /D:/Programmation/Python/Projets de ouf/ChampSelectPrediction/getmatches.ipynb
m = lol_watcher.league.masters_by_queue(REGION, "RANKED_SOLO_5x5")
gm = lol_watcher.league.grandmaster_by_queue(REGION, "RANKED_SOLO_5x5")
c = lol_watcher.league.challenger_by_queue(REGION, "RANKED_SOLO_5x5")
players_dict: dict = json.load(open("players.json", "r"))

with tqdm(total=len(m["entries"] + gm["entries"] + c["entries"])) as pbar:
    for player in m["entries"] + gm["entries"] + c["entries"]:
        if not player["summonerName"]:
            print("No summoner name")
            continue
        if player["summonerName"] in players_dict:
            print("Already in")
            continue
        summoner = lol_watcher.summoner.by_name(REGION, player["summonerName"])
        players_dict[player["summonerName"]] = summoner["puuid"]
        json.dump(players_dict, open("players.json", "w"))
        pbar.update(1)


## Find the last 5 games for each master grandmaster and challenger player

In [7]:
from tqdm import tqdm

players_dict = json.load(open("players.json", "r"))
games_dict = json.load(open("games_ids.json", "r"))

with tqdm(total=len(players_dict)) as pbar:
    for player_name, player_puid in players_dict.items():
        if player_name in games_dict:
            print("Already in")
            continue

        matches = list(
            lol_watcher.match.matchlist_by_puuid(REGION, player_puid, count=5)
        )
        if not matches:
            print("No matches")
            continue
        games_dict[player_name] = matches
        json.dump(games_dict, open("games_ids.json", "w"))
        pbar.update(1)

print("Done")

  9%|▉         | 285/3182 [05:18<53:55,  1.12s/it]   


KeyboardInterrupt: 

## Extract champions used in each match and the result of the match

In [ ]:
# TODO
from pandas import DataFrame

games_players_dict = json.load(open("games_ids.json", "r"))
games = DataFrame()

for player_name, games_ids in games_players_dict.items():
    for game_id in games_ids:
        if game_id in games_dict:
            print("Already in")
            continue
        game = lol_watcher.match.by_id(REGION, game_id)
        games_dict[game_id] = game
        json.dump(games_dict, open("games.json", "w"))